[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/EMQA_oil_analysis/EMQA_oil_analysis.ipynb)

# EMQA_oil_analysis

Brent crude oil historical analysis (2015-2025).

**Output:** `oil_prices_historical.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
brent = fetch('BZ=F', '2015-01-01', '2025-12-31')

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(brent.index, brent.values, color=COLORS['blue'], lw=0.8)
ax.axhline(brent.mean(), color='gray', ls='--', alpha=0.5, lw=0.8)

events = [('2020-04-20', 'COVID\nCrash', -30), ('2022-03-08', 'Ukraine\nWar', 20)]
for date, label, offset in events:
    idx = pd.Timestamp(date)
    if idx in brent.index:
        val = brent.loc[idx]
    else:
        val = brent.iloc[brent.index.get_indexer([idx], method='nearest')[0]]
    ax.annotate(label, xy=(idx, val), xytext=(0, offset), textcoords='offset points',
                ha='center', fontsize=9, color=COLORS['red'],
                arrowprops=dict(arrowstyle='->', color=COLORS['red'], lw=1.2))

ax.set_title('Brent Crude Oil Prices (2015-2025)', fontweight='bold', fontsize=13)
ax.set_ylabel('USD/bbl')
ax.legend(['Brent Price', 'Mean'], loc='upper center', bbox_to_anchor=(0.5, -0.10), ncol=2, frameon=False)
fig.tight_layout()
save_fig(fig, 'oil_prices_historical.pdf')
plt.show()
